<a href="https://colab.research.google.com/github/SukheshNuthalapati/BeeSnap-ML/blob/master/Fake_News_Labeler_Fall_2022_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Fake News Labeler

This notebook contains code to label the FNC-Core dataset with a team of pre-trained experts. 

## Dataset

The FNC (Fake-News-Covid) dataset is a twitter-sampled stream of tweets for fake news detection.It is collected along with its social context of usernames, account details, retweets, likes, quote tweets, among other features.

FNC has several components:

- **FNC-Raw**: The raw stream collected from the Twitter Sampled Stream
- **FNC-Filtered**: The filtered stream obtained from the Twitter Sampled Stream with covid-related keyword filters.
- **FNC-Neighbors**: Samples from FNC-Raw that were not filtered into FNC-Filtered due to missing, censored, or mispelled keywords, but are semantically similar to FNC-Filtered. We detect these with embedding extensions.
- **FNC-Extended**: FNC-Filtered combined with FNC-Neighbors to yield a more complete filtered dataset. We can refer to monthly subsets as FNC-Extended-\<MonthYear\>, e.g. FNC-Extended-Jan2020

These are the unlabeled parts of FNC. With this notebook, we can label a subset of FNC-Extended with high accuracy. These also fall under several components:

- **FNC-Extended-Oracle**: Samples from FNC-Extended that are labeled with human annotators. FNC-Extended-Oracle has 10k samples. Since FNC-Extended spans 25 months, we have 400 labeled samples per month.
- **FNC-Expert-\<Expert\>-\<MonthYear\>**: The FNC-Extended-\<MonthYear\> subset is labeled by an expert with name \<Expert\>.
- **FNC-Labeled**: We integrate labels from several experts to generate the final labeled set. We can evaluate labeling accuracy on the intersection with FNC-Extended-Oracle. Monthly subsets are FNC-Labeled-\<MonthYear\>

## This Notebook
In this notebook, we will:

- Generate FNC-Filtered (unless the raw data is already of this format...should recheck)
- Possibly generate FNC-Neighbors (unless raw data is already of this format...should recheck)
- Generate FNC-Expert. For this, we have a list of experts already generated. We will use EdnaML Deployments to load an expert, then load our data from Azure, then label each point, and save the generated label. If a model abstains for some point, we will score that as a [-1]. True News is [1]. Fake News is [0]. Then, the labels will be saved to this instance. We will download the labels, labeled as FNC-Expert-[ExpertName]-[MonthYear]. For each file, we will also save any necessary label characteristics in the same line, i.e.: [label,L-score, model-l-score-threshold, etc]
- Possibly Generate FNC-Labeled. Once we have several expert labeled sets, we will integrate them together using Snorkel, EEWS, etc... Snorkel is easy to implement. We might skip EEWS and ATEAM.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 8.4 MB/s 
     |████████████████████████████████| 163 kB 58.6 MB/s 
     |████████████████████████████████| 7.6 MB 55.9 MB/s 


# 1. Setup -- Gdrive Connect, Git clone, Model  Downloads

## Connect to Google Drive

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
import os.path as osp
import os, glob, shutil

In [ ]:
# Set Gogle Drive Connection
#if not osp.exists("./drive"):
from google.colab import drive

#https://stackoverflow.com/questions/69822304/google-colab-google-drive-can%c2%b4t-be-mounted-anymore-browser-popup-google-dri
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Git Clone

### From Source

In [ ]:
! rm -rf -- GLAMOR

In [ ]:
! git clone -b master https://github.com/asuprem/GLAMOR

Cloning into 'GLAMOR'...
remote: Enumerating objects: 7326, done.
remote: Counting objects: 100% (1812/1812), done.
remote: Compressing objects: 100% (539/539), done.
remote: Total 7326 (delta 1122), reused 1743 (delta 1090), pack-reused 5514
Receiving objects: 100% (7326/7326), 2.06 MiB | 13.97 MiB/s, done.
Resolving deltas: 100% (4823/4823), done.


In [ ]:
!pip install -e GLAMOR/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/GLAMOR
     |████████████████████████████████| 1.3 MB 9.7 MB/s 
  Running setup.py develop for ednaml


###  From PyPi

In [ ]:
#! python -V

In [ ]:
#! pip3 install --pre ednaml==0.1.4

## AlBERT / ModelFile Data Download

(And assume they are FNC-Extended-MonthYear)

In [ ]:
!cp ./drive/MyDrive/Datasets/PreTrained/Albertv2/30k* .
!cp ./drive/MyDrive/Datasets/PreTrained/Albertv2/pytorch_model.bin .

# Restart Runtime Here!!!!

# 2. Raw Data

In [ ]:
tweet_file = "tweets-2020-01-22"
proxy = 20

# 3. Generating FNC-Filtered from FNC-Raw

Here, we will use an EdnaML workflow to generate FNC-Filtered from FNC-Raw. This is not really a learnable model, rather a simple keyword matcher.

Essentially, we will build a workflow where we:\

1. Crawl an FNC Source and download if it does not exist
2. Create datashards from the Source, with only testing data since there is no "training" to be performed
3. Deploy a ModelAbstract model that reads a batch of raw text data, and checks whether elements of this batch contain our keywords. If they do, we label 1. If they do not, we label 0. 
4. The Deployment thus generates 2 outputs: a file containing fnc-filtered, and a file fnc-nonfiltered

In [ ]:
# cleanup
!rm -rf -- test-datashard-artifacts

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

'1.12.1+cu113'

In [ ]:
config = "./GLAMOR/profiles/FNC/filtered/config.yml"
crawler_generator = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filter_model_deployment = "./GLAMOR/profiles/FNC/fnc-filtered.py"

In [ ]:
ed = EdnaDeploy(config=config)
ed.cfg.DEPLOYMENT.OUTPUT_ARGS["basename"] = "%s"%tweet_file
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["azfile"] = "%s.json.gz"%tweet_file

In [ ]:
ed.add(crawler_generator)
ed.add(filter_model_deployment)
ed.apply()

04:30:36 Adding a crawler, from /content/GLAMOR/profiles/FNC/fnc-crawler.py, with inferred name FNCCrawler
04:30:36 Adding a generator, from /content/GLAMOR/profiles/FNC/fnc-crawler.py, with inferred name FNCRawGenerator
04:30:36 Adding a model, from /content/GLAMOR/profiles/FNC/fnc-filtered.py, with inferred name FNCFilter
04:30:36 Adding a deployment, from /content/GLAMOR/profiles/FNC/fnc-filtered.py, with inferred name FNCFilterDeployment
04:30:36 ****************************************
04:30:36 
04:30:36 
04:30:36 Using the following configuration:
04:30:36 DEPLOYMENT:
  DATAREADER:
    CRAWLER_ARGS:
      azcontainer: edna-covid-raw
      azfile: tweets-2020-01-22.json.gz
      azstorage: ednadatasets
    DATAREADER: AlbertReader
    DATASET_ARGS:
      classificationclass:
      - fnews
      shard_replace: false
      shardname: fnc-raw-shard
      shardpath: datashard-artifacts
      shardsize: 20000
      shuffle: false
    GENERATOR: null
    GENERATOR_ARGS: {}
  DEPLOY: Bas

Registering a crawler: <class './GLAMOR/profiles/FNC/fnc-crawler.py.FNCCrawler'>, from file: /content/GLAMOR/profiles/FNC/fnc-crawler.py
Registering a generator: <class './GLAMOR/profiles/FNC/fnc-crawler.py.FNCRawGenerator'>, from file: /content/GLAMOR/profiles/FNC/fnc-crawler.py
Registering a model: <class './GLAMOR/profiles/FNC/fnc-filtered.py.FNCFilter'>, from file: /content/GLAMOR/profiles/FNC/fnc-filtered.py
Registering a deployment: <class './GLAMOR/profiles/FNC/fnc-filtered.py.FNCFilterDeployment'>, from file: /content/GLAMOR/profiles/FNC/fnc-filtered.py
5637377/5637377 bytes [████████████████████████████████████████████████████████████████████████████████████████████████████]
Download of tweets-2020-01-22.json.gz to https://ednadatasets.blob.core.windows.net/edna-covid-raw/tweets-2020-01-22.json.gz completed


04:30:39 Generating dataloader `FNCRawGenerator` with `test` mode
04:30:47 Generated test data/query generator
04:30:47 Loaded ednaml_model_builder from ednaml.models to build model
04:30:47 Finished instantiating model with FNCFilter architecture
04:30:47 Adding plugins after constructing model
04:30:47 No saved model weights provided. Inferring weights path.
04:30:47 No previous stop exists. Not loading weights.
04:30:47 Model Summary retured the following error:
04:30:47 Traceback (most recent call last):
  File "/content/GLAMOR/src/ednaml/core/EdnaML.py", line 875, in getModelSummary
    self.cfg.TRAIN_TRANSFORMATION.INPUT_SIZE,
AttributeError: 'TransformationConfig' object has no attribute 'INPUT_SIZE'

04:30:47 Multi-gpu or non-gpu not yet fully supported.


NOT saving metadata. saveMetadata() function not set up.


In [ ]:
ed.deploy()

04:30:47 Starting deployment
04:30:47 Logging to:	fnc-filter-v1-offshelf-raw-logger.log
04:30:47 Setting up plugin hooks. Plugins will fire during:  always
04:30:47 Executing deployment for  1 epochs
04:30:47 Starting epoch 0
04:30:49 Executing end of epoch steps
04:30:49 Completed deployment task.


## Cleanup

In [ ]:
import os
if os.path.exists(tweet_file+".json"):
  os.remove(tweet_file+".json")
if os.path.exists(tweet_file+".json.gz"):
  os.remove(tweet_file+".json.gz")
! rm test-datashard-artifacts/fnc-raw*
! rm train-datashard-artifacts/fnc-raw*
! rm test-datashard-artifacts/len.txt

In [ ]:
! wc -l $tweet_file-filtered.json

17718 tweets-2020-01-22-filtered.json


In [ ]:
! wc -l $tweet_file-unfiltered.json

6333 tweets-2020-01-22-unfiltered.json


## Filtered subset scaling (for class)

In [ ]:
filter_scale = 1    # For Caden, Sneh --> change this to 0.8
unfilter_scale = 1  # For Caden, Sneh --> change this to 0.6

### Scaling

In [ ]:
import os
tf_filtered = tweet_file + "-filtered.json"
tf_unfiltered = tweet_file + "-unfiltered.json"
# Set reduction scale
import random
random.seed(34598344)
# Reduce filtered.json
filt_obj = open(tf_filtered, "r")
filtout_obj = open(tf_filtered + "2", "w")
for row in filt_obj:
    if random.random() <= filter_scale:
      filtout_obj.write(row)
filtout_obj.close()
filt_obj.close()
# Reduce unfiltered.json
unfilt_obj = open(tf_unfiltered, "r")
unfiltout_obj = open(tf_unfiltered + "2", "w")
for row in unfilt_obj:
    if random.random() <= unfilter_scale:
      unfiltout_obj.write(row)
unfiltout_obj.close()
unfilt_obj.close()
# Delete old files
os.remove(tf_filtered)
os.remove(tf_unfiltered)
# Rename to old file name
os.rename(tf_filtered+"2", tf_filtered)
os.rename(tf_unfiltered+"2", tf_unfiltered)

In [ ]:
! wc -l $tweet_file-filtered.json

17718 tweets-2020-01-22-filtered.json


In [ ]:
! wc -l $tweet_file-unfiltered.json

6333 tweets-2020-01-22-unfiltered.json


# 4. Generating FNC-Neighbors from FNC-Filtered and FNC

Here, we will take the json files generated from FNC-Filter: `filter.json` and `unfilter.json`.

The filter.json content is used to fine-tune a BERT model. This BERT model implements keyword masking, that is, specific keywords are masked

Then we use a cluster_estimator plugin to estimate the best number of clusters with KMeans for the training dataset.

Then, we insert a high-density estimator plugin with the best number of clusters as proxies

Then, we will use a Deployment to iterate through unfilter. For each, we will generate the features, and use the high-density estimator plugin to check if it exists in the high density set of existing clusters. Note that the unfilter data implements the same masking as the training model. Any samples that exist are placed in `extended.json`

## 4.1 Finetuning MLM with keyword masking

In [ ]:
# Cleanup ONLY if starting from scratch, not if already doing 4.1
# Also cleanup if switching datasets
!rm test-datashard-artifacts/fnc-filtermask*
!rm train-datashard-artifacts/fnc-filtermask*

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

'1.12.1+cu113'

In [ ]:
config = "./GLAMOR/profiles/FNC/extension/config.yml"
crawler = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filtermask_generator = "./GLAMOR/profiles/FNC/fnc-filtermasked.py"
extension_model = "./GLAMOR/profiles/FNC/fnc-extension.py"

In [ ]:
eml = EdnaML(config=config, config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file), ("SAVE.MODEL_VERSION", 1), ("SAVE.STEP_SAVE_FREQUENCY", 50000), ("EXECUTION.EPOCHS", 10)])
#eml = EdnaML(config=config, config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file)])
eml.cfg.EXECUTION.DATAREADER.CRAWLER_ARGS["train_file"] = "%s-filtered.json"%tweet_file
eml.cfg.EXECUTION.DATAREADER.CRAWLER_ARGS["test_file"] = "%s-unfiltered.json"%tweet_file

Injected key-value pair:  SAVE.MODEL_QUALIFIER, tweets-2020-01-22
Injected key-value pair:  SAVE.MODEL_VERSION, 4
Log file exists at fnc-extension-v4-albert-tweets-2020-01-22/fnc-extension-v4-albert-tweets-2020-01-22-logger.log. Will attempt to append there.


In [ ]:
eml.add(crawler)
eml.add(filtermask_generator)
eml.add(extension_model)
eml.apply()

19:49:02 Adding a crawler, from /content/GLAMOR/profiles/FNC/fnc-crawler.py, with inferred name FNCCrawler
19:49:02 Adding a generator, from /content/GLAMOR/profiles/FNC/fnc-crawler.py, with inferred name FNCRawGenerator
19:49:02 Adding a generator, from /content/GLAMOR/profiles/FNC/fnc-filtermasked.py, with inferred name FNCFilterMaskGenerator
19:49:02 Adding a model, from /content/GLAMOR/profiles/FNC/fnc-extension.py, with inferred name FNCAlbertModeler
19:49:02 Adding a trainer, from /content/GLAMOR/profiles/FNC/fnc-extension.py, with inferred name FNCAlbertModelerTrainer
19:49:02 ****************************************
19:49:02 
19:49:02 
19:49:02 Using the following configuration:


Registering a crawler: <class './GLAMOR/profiles/FNC/fnc-crawler.py.FNCCrawler'>, from file: /content/GLAMOR/profiles/FNC/fnc-crawler.py
Registering a generator: <class './GLAMOR/profiles/FNC/fnc-crawler.py.FNCRawGenerator'>, from file: /content/GLAMOR/profiles/FNC/fnc-crawler.py
Registering a generator: <class './GLAMOR/profiles/FNC/fnc-filtermasked.py.FNCFilterMaskGenerator'>, from file: /content/GLAMOR/profiles/FNC/fnc-filtermasked.py
Registering a model: <class './GLAMOR/profiles/FNC/fnc-extension.py.FNCAlbertModeler'>, from file: /content/GLAMOR/profiles/FNC/fnc-extension.py
Registering a trainer: <class './GLAMOR/profiles/FNC/fnc-extension.py.FNCAlbertModelerTrainer'>, from file: /content/GLAMOR/profiles/FNC/fnc-extension.py
Registering a deployment: <class './GLAMOR/profiles/FNC/fnc-extension.py.FNCTrainingFeaturesDeploy'>, from file: /content/GLAMOR/profiles/FNC/fnc-extension.py


19:49:02 DEPLOYMENT:
  DATAREADER:
    CRAWLER_ARGS: {}
    DATAREADER: DataReader
    DATASET_ARGS: {}
    GENERATOR: null
    GENERATOR_ARGS: {}
  DEPLOY: BaseDeploy
  DEPLOYMENT_ARGS: {}
  EPOCHS: 1
  OUTPUT_ARGS: {}
  PLUGIN:
    HOOKS: always
    RESET: false
EXECUTION:
  DATAREADER:
    CRAWLER_ARGS:
      azcontainer: edna-covid-raw
      azfile: tweets-2020-01-22.json.gz
      azstorage: ednadatasets
      test_file: tweets-2020-01-22-unfiltered.json
      train_file: tweets-2020-01-22-filtered.json
    DATAREADER: AlbertReader
    DATASET_ARGS:
      classificationclass:
      - fnews
      data_shuffle: true
      keyword_mask: true
      keywords:
      - covid
      - corona
      - mask
      - wuhan
      - n95
      - sars
      - monkey
      - pandemic
      - social
      - quarantin
      - virus
      - infect
      - lock
      - ppe
      - variant
      - vaccine
      - travel
      - omicron
      - ivermectin
      - plandemic
      - 5g
      - gates
      - 

loading weights file pytorch_model.bin


19:49:20 Finished instantiating model with FNCAlbertModeler architecture
19:49:20 Adding plugins after constructing model
19:49:20 No saved model weights provided.


Errors 
	 {'missing_keys': [], 'unexpected_keys': [], 'error_msgs': []}


19:49:22 Model Summary retured the following error:
19:49:22 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torchinfo/torchinfo.py", line 287, in forward_pass
    _ = model.to(device)(*x, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1148, in _call_impl
    result = forward_call(*input, **kwargs)
  File "/content/GLAMOR/src/ednaml/models/ModelAbstract.py", line 141, in forward
    x, **kwargs
  File "./GLAMOR/profiles/FNC/fnc-extension.py", line 110, in forward_impl
    head_mask=head_mask,
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1148, in _call_impl
    result = forward_call(*input, **kwargs)
  File "/content/GLAMOR/src/ednaml/models/Albert.py", line 706, in forward
    input_ids, position_ids=position_ids, token_type_ids=token_type_ids
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1148, in _call_impl
    result = forward_call(*inpu

INPUT SIZE ====  [16, 512]
NOT saving metadata. saveMetadata() function not set up.


In [ ]:
eml.train()

19:49:22 Starting training
19:49:22 Logging to:	fnc-extension-v4-albert-tweets-2020-01-22-logger.log
19:49:22 Models will be saved to local directory:	fnc-extension-v4-albert-tweets-2020-01-22
19:49:22 Models will be backed up to drive directory:	./drive/MyDrive/Projects/FNC/Models/fnc-extension-v4-albert-tweets-2020-01-22
19:49:22 Models will be saved with base name:	fnc-extension-v4_epoch[].pth
19:49:22 Optimizers will be saved with base name:	fnc-extension-v4_epoch[]_optimizer.pth
19:49:22 Schedulers will be saved with base name:	fnc-extension-v4_epoch[]_scheduler.pth
19:49:22 Resuming training from epoch 1. Loading saved state from 0
19:49:22 Loading model, optimizer, and scheduler from drive backup.
19:49:22 Finished loading model state_dict from ./drive/MyDrive/Projects/FNC/Models/fnc-extension-v4-albert-tweets-2020-01-22/fnc-extension-v4_epoch0.pth
19:49:22 Finished loading optimizer state_dict from ./drive/MyDrive/Projects/FNC/Models/fnc-extension-v4-albert-tweets-2020-01-22/fn

KeyboardInterrupt: ignored

In [ ]:
eml.trainer.save()

19:51:04 Saving model, optimizer, and scheduler.
19:51:05 Performing drive backup of model, optimizer, and scheduler.


In [ ]:
eml.eval()

## 4.1.1. Deployment to generate training features
Now we will use our trained LLM to generate the training features again.

These features will be used to determine the best number of clusters and other hyperparameters. Subsequently, we can use our calculated hyperparameters to build a K-Means Clustering Plugin that includes a High-Density Set estimator.

In [ ]:
! rm *.h5*

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
config = "./GLAMOR/profiles/FNC/extension/config.yml"
deploy = "./GLAMOR/profiles/FNC/extension/training_features.yml"  # remove masking in data generation process
crawler = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filtermask_generator = "./GLAMOR/profiles/FNC/fnc-filtermasked.py"
extension_model = "./GLAMOR/profiles/FNC/fnc-extension.py"

In [ ]:
ed = EdnaDeploy(config=config, deploy = deploy, dataloader_mode = "train", config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file)])
ed.cfg.DEPLOYMENT.OUTPUT_ARGS["feature_file"] = "%s-filtered-features"%tweet_file
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["train_file"] = "%s-filtered.json"%tweet_file

In [ ]:
ed.add(crawler)
ed.add(filtermask_generator)
ed.add(extension_model)  # With model AND deployment defined
ed.apply()

In [ ]:
ed.deploy()

## 4.2 Estimating best number of clusters from the BERT features

In [ ]:
def cluster_sweep(features_file, batch_size, max_iters, cluster_range):
  import h5py, time
  from sklearn.cluster import MiniBatchKMeans
  inertia = []
  data = h5py.File(features_file, 'r')
  data_size = data['features'].shape[0]
  
  for k in cluster_range:
    print("Starting sweep for k={kval}, with {iters} iterations".format(kval=k, iters=max_iters))
    kmeans = MiniBatchKMeans(n_clusters = k, random_state = 23465356, batch_size = batch_size)
    stime = time.time()
    for iters in range(max_iters):
      for i in range(0, data_size, batch_size):
          current_data = data['features'][i:i+batch_size]
          kmeans.partial_fit(current_data)
      if iters%5 == 0:
        etime = round(time.time() - stime, 2)
        print("\t[{elapse} s] -- Completed {iters} iterations".format(iters=iters, elapse = etime))
        stime = time.time()
    print("\tCompeted MBKM for k={kval}, with inertia: {inertia}".format(kval=k, inertia = kmeans.inertia_))
    inertia.append(kmeans.inertia_)
  return inertia

In [ ]:
cluster_range = [10,20,50]
inertia = cluster_sweep(
    features_file = "%s-filtered-features.h5"%tweet_file,
    batch_size=256,
    max_iters = 25,
    cluster_range = cluster_range
)

In [ ]:
# In case of interruptions or crashes...
#inertia2 = [1660, 5510, 3100, 2290] + inertia

In [ ]:
import seaborn as sns
import pandas as pd
sns.set_theme(style="darkgrid")
sns.lineplot(x="cluster_range", y="inertia", data=pd.DataFrame(list(zip(inertia, cluster_range)), columns=["inertia", "cluster_range"]), 
             palette="tab10", linewidth=2.5, marker="o", markersize=10). \
              set(title="Elbow Plot for KMeans")

So, we find here that for FNC Jan 2020 that **20 PROXIES** fits the elbow visual test.

In [ ]:
proxy = 20

## 4.3 Generate Plugins 

Using k=20 proxies, we will now generate the FastKMeans Plugin for our LLM for this FNC window. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
config = "./GLAMOR/profiles/FNC/extension/config.yml"
deploy = "./GLAMOR/profiles/FNC/extension/fastkmeans.yml" #basically, set up plugin info, and remove masking
crawler = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filtermask_generator = "./GLAMOR/profiles/FNC/fnc-filtermasked.py"
extension_model = "./GLAMOR/profiles/FNC/fnc-extension.py"
fastkmeans = "./GLAMOR/profiles/FNC/fnc-fastkmeans.py"  # contains plugin and deployment for the plugin

In [ ]:
ed = EdnaDeploy(config=config, deploy = deploy, dataloader_mode = "train", config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file), ("SAVE.SAVE_FREQUENCY", 1)])
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["train_file"] = "%s-filtered.json"%tweet_file
ed.cfg.MODEL_PLUGIN["FastKMP-l2"].PLUGIN_KWARGS["proxies"] = proxy

In [ ]:
ed.add(crawler)
ed.add(filtermask_generator)
ed.add(extension_model)  # With model AND deployment defined
ed.add(fastkmeans)  # # With plugin AND deployment defined; deployment replaces prior
ed.apply()

In [ ]:
ed.deploy()

## 4.4 Apply Plugin to Generate Neighbors

Now we will use our generated plugin to determine which members of fns-unfiltered should belong in fnc-filtered because they are neighbors.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
config = "./GLAMOR/profiles/FNC/extension/config.yml"
deploy = "./GLAMOR/profiles/FNC/extension/neighbors.yml" #basically, set up plugin info, and remove masking
crawler = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filtermask_generator = "./GLAMOR/profiles/FNC/fnc-filtermasked.py"
extension_model = "./GLAMOR/profiles/FNC/fnc-extension.py"
fastkmeans = "./GLAMOR/profiles/FNC/fnc-fastkmeans.py"  # contains plugin and deployment for the plugin
neighbors = "./GLAMOR/profiles/FNC/fnc-neighbors.py"  # contains deployment for neighbors generation

In [ ]:
ed = EdnaDeploy(config=config, deploy = deploy, config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file), ("SAVE.DRIVE_BACKUP", True)])
ed.cfg.DEPLOYMENT.OUTPUT_ARGS["basename"] = "%s"%tweet_file  # deployment will generate twt-neighbors.json
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["test_file"] = "%s-unfiltered.json"%tweet_file
ed.cfg.MODEL_PLUGIN["FastKMP-l2"].PLUGIN_KWARGS["proxies"] = proxy

In [ ]:
ed.add(crawler)
ed.add(filtermask_generator)
ed.add(extension_model)  # With model AND deployment defined
ed.add(fastkmeans)  # # With plugin AND deployment defined; deployment replaces prior
ed.add(neighbors)
ed.apply()

In [ ]:
ed.deploy()

## 4.5. Generating FNC-Extended = FNC-Filtered + FNC-Neighbors

Here, we combine fnc-filtered.json and fnc-neighbors.json to create fnc-extended.json. Then we upload this to our azure blob for this month!

In [ ]:
original = sum(1 for line in open("%s.json"%tweet_file))
filtered = sum(1 for line in open("%s-filtered.json"%tweet_file))
unfiltered = sum(1 for line in open("%s-unfiltered.json"%tweet_file))
neighbor = sum(1 for line in open("%s-neighbor.json"%tweet_file))
extension = 100 * neighbor / filtered
lift = 100 * neighbor / unfiltered
print(
"""
-------- STATISTICS ------------

ORIGINAL    {original}
FILTERED    {filtered}
UNFILTERED  {unfiltered}
NEIGHBORS   {neighbor}

EXTENSION   {extension}%
LIFT        {lift}%
""".format(original=original, filtered=filtered, unfiltered=unfiltered, neighbor=neighbor, extension=round(extension,2), lift=round(lift,2)))

In [ ]:
cat "$tweet_file-filtered.json" "$tweet_file-neighbor.json" >> "$tweet_file-extended.json"

## 4.6. Generate FNC-Extended-Oracle-Unlabeled, and upload

Here, we will randomly select ~500 points to perform manual labeling. We will try to select these points from each cluster of our FNC LLM Model.

Specifically, we will first use our existing plugin to compute the distances of each point in the extension to its cluster center, as well as its cluster index (the index itself is not semantic). 

After the deployment is complete, we will sweep across the data to find, for each cluster, the closest point, as well as some points further away to generate a more complete representation. That is, since we have 20 clusters (in FNC Jan), and require 500 points, for each cluster we can bin  into 24 regions + closest, and from the 24 regions, select one point at random.

In [ ]:
#!wget https://ednadatasets.blob.core.windows.net/edna-covid-extended/$tweet_file-extended.json.gz
#!gunzip $tweet_file-extended.json.gz

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
config = "./GLAMOR/profiles/FNC/extension/config.yml"
deploy = "./GLAMOR/profiles/FNC/extension/oracle.yml" #basically, set up plugin info, and remove masking; shuffle, and set up the inputs for oracle file generation
crawler = "./GLAMOR/profiles/FNC/fnc-crawler.py"
filtermask_generator = "./GLAMOR/profiles/FNC/fnc-filtermasked.py"
extension_model = "./GLAMOR/profiles/FNC/fnc-extension.py"
fastkmeans = "./GLAMOR/profiles/FNC/fnc-fastkmeans.py"  # contains plugin and deployment for the plugin
oracle = "./GLAMOR/profiles/FNC/fnc-oracle.py"  # contains deployment for oracle selection

In [ ]:
ed = EdnaDeploy(config=config, deploy = deploy, config_inject=[("SAVE.MODEL_QUALIFIER", tweet_file), ("SAVE.DRIVE_BACKUP", True)])
ed.cfg.DEPLOYMENT.OUTPUT_ARGS["basename"] = "%s"%tweet_file  
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["test_file"] = "%s-extended.json"%tweet_file
ed.cfg.MODEL_PLUGIN["FastKMP-l2"].PLUGIN_KWARGS["proxies"] = proxy

In [ ]:
ed.add(crawler)
ed.add(filtermask_generator)
ed.add(extension_model)  # With model AND deployment defined
ed.add(fastkmeans)  # # With plugin AND deployment defined; deployment replaces prior
ed.add(oracle)
ed.apply()

In [ ]:
ed.deploy()

## 4.7 Upload to azure

https://docs.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?tabs=environment-variable-windows

In [ ]:
!gzip $tweet_file-oracle.json

In [ ]:
!gunzip $tweet_file-extended.json.gz

In [ ]:
!gzip $tweet_file-extended.json

In [ ]:
!pip install azure-storage-blob

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
connect_str = "DefaultEndpointsProtocol=https;AccountName=ednadatasets;AccountKey=7pZKgsFLvckW3933BSOIzRvarSisKeDSCT5E/dXNs7vlfEEHG5MworeL2VoyA14pcZXoCaHNIR5r+AStqZQGaQ==;EndpointSuffix=core.windows.net"
# Create the BlobServiceClient object which will be used to create a container client
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

extended_file = "%s-extended.json.gz"%tweet_file
oracle_file = "%s-oracle.json.gz"%tweet_file

In [ ]:
# Create a unique name for the container
container_name = "edna-covid-extended"
blob_client = blob_service_client.get_blob_client(container=container_name, blob=extended_file)
with open(extended_file, "rb") as data:
    blob_client.upload_blob(data)

In [ ]:
# Create a unique name for the container
container_name = "edna-covid-extended-oracle"
blob_client = blob_service_client.get_blob_client(container=container_name, blob=oracle_file)
with open(oracle_file, "rb") as data:
    blob_client.upload_blob(data)

# 5. Generate Labels from Experts

Here, we will label FNC-Extended for a month with models from ENFD, Fakeddit, and NELA's Bert-variant models. For now, we will use the Albert-Base-v2 models only.

For each classifier, we will generate a text file with labels for each entry. Since there is no data shuffling, labels across text files will sync up. We will then upload these saved files to our Drive location for ease of access and future integration.

We need the following outputs in each text file:
  - **Predicted label.** 0 for True news, 1 for Fake news.
  - **Distance-to-proxy (l2)**. Raw distance in embedding space to the nearest proxy.
  - **Distance-to-proxy (cos)**. Cosine distance
  - **High density set**. 0 or 1. Whether sample is in the high density set.
  - **High density label**. The ground truth label of the corresponding high-density set cluster. This is the same as the proxy label, or nearest proxy label.
  - **L-Score**. Smoothness score for the input using perturbations. Probably take the longest time.
  - **L-Threshold**. The threshold for the nearest proxy / high density set.
  - **Logit-confidence**. The raw logit probability of the predicted class.
  - **Logit-average**. The average logit probability of the predicted class.

In [ ]:
# See FNC Experts (And Masking!) notebook
tweet_file = "tweets-2020-01-22"

# efnd | nela | fakeddit
dataset = "efnd"    

# (ONLY FOR efnd; OTHERS YOU CAN IGNORE) 
# cmu_miscov19 | kagglefn_short | kagglefn_long | cov19_fn_title | cov19_fn_text | coaid_news | cov_rumor | covid_fn | covid_cq
subdataset = "kagglefn_short"

# nomask | rtm | rwm | ktm_tfidf | kwm_tfidf | ktm_att | kwm_att | ktrtm_tfidf | kwrwm_tfidf | ktrtm_att | kwrwm_att
masking = "nomask"  

# albert-base-v2
model = "albert-base-v2"

## Applying

In [ ]:

#Select the correct model-file!
model_variant = "./GLAMOR/profiles/FNC/experts/configs/model-bertvariant.yml" 

# plugin deploy file and proxies
plugin_deploy_generation_file = "./GLAMOR/profiles/FNC/experts/plugin_generation.py" 
plugin_proxies = {"efnd-cmu_miscov19": 20, "efnd-kagglefn_short": 40, "efnd-kagglefn_long": 20,
                  "efnd-cov19_fn_title": 30, "efnd-cov19_fn_text": 35, "efnd-coaid_news": 50,
                  "efnd-cov_rumor": 30, "efnd-covid_fn": 30, "efnd-covid_cq": 50, 
                  "nela": 40, "fakeddit": 30}
if dataset == "efnd":
  proxy = plugin_proxies[dataset + "-" + subdataset]
else:
  proxy = plugin_proxies[dataset]

if dataset == "efnd":
  #----------------------------- EFND ------------------------------------
  dataset_args = {
      "data_folder" : "Data",
      "include": [subdataset]
  }
  model_qualifier = subdataset
elif dataset == "nela":
  #----------------------------- NELA ------------------------------------
  dataset_args = {
      "data_folder" : "Data",
      "sub_folder" : "nela-covid-2020"
  }
  model_qualifier = "nela_covid_2020"
elif dataset == "fakeddit":
  #----------------------------- FAKEDDIT ------------------------------------
  dataset_args = {
      "data_folder" : "Data"
  }
  model_qualifier = "fakeddit"
else:
  raise NotImplementedError()

# nomask | rtm | rwm | ktm_tfidf | kwm_tfidf | ktm_att | kwm_att | ktrtm_tfidf | kwrwm_tfidf | ktrtm_att | kwrwm_att
mask_overall = True;tm = False;wm = False;ktm=False;kwm=False;
if masking == "nomask":
  mask_overall = False
if masking == "rtm" or masking == "ktrtm_tfidf" or masking == "ktrtm_att":
  tm = True
if masking == "rwm" or masking == "kwrwm_tfidf" or masking == "kwrwm_att":
  wm = True
if masking == "ktm_tfidf" or masking == "ktm_att" or masking == "ktrtm_att":
  ktm = True
if masking == "kwm_tfidf" or masking == "kwm_att" or masking == "kwrwm_att":
  kwm = True

# Other options
model_core_name = "-".join(["fnc","expert",dataset, masking])
model_backbone = model
model_base = model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import ednaml, torch
from ednaml.core import EdnaDeploy

In [ ]:
from ednaml.deploy.HFDeploy import HFDeploy
import os

class LabelingDeployment(HFDeploy):
  def output_setup(self, **kwargs):
    self.ofile = kwargs.get("label_file_name")
    if self.ofile == "None":
      raise ValueError("No filename provided")
    self.oobj = open(self.ofile, "w")
    self.oobj.write(",".join([
      "predicted_label",
      "l2_dist",
      "l2_hdthreshold",
      "l2_proxylabels",
      "cos_dist",
      "cos_hdthreshold",
      "cos_proxylabels",
      "l_score",
      "smooth_l_score",
      "l_threshold",
      "smooth_l_threshold",
      "l_proxylabel",
      "logit_raw",
      "logit_threshold"
      ])+"\n")

  def output_step(self, logits, features, secondary):
    predicted_label = torch.argmax(torch.nn.functional.softmax(logits.cpu(), dim=1), dim=1).tolist()
    l2_dist = secondary[2]["FastKMP-l2"]["distance"].to(torch.float32).tolist()
    l2_hdthreshold = secondary[2]["FastKMP-l2"]["threshold"].to(torch.float32).tolist()
    l2_proxylabels = secondary[2]["FastKMP-l2"]["proxy_labels"].to(torch.float32).tolist()
    
    cos_dist = secondary[2]["FastKMP-cos"]["distance"].to(torch.float32).tolist()
    cos_hdthreshold = secondary[2]["FastKMP-cos"]["threshold"].to(torch.float32).tolist()
    cos_proxylabels = secondary[2]["FastKMP-cos"]["proxy_labels"].to(torch.float32).tolist()

    l_score = secondary[2]["FRL-midas"]["l_score"]
    smooth_l_score = secondary[2]["FRL-midas"]["smooth_l_score"]
    l_threshold = secondary[2]["FRL-midas"]["l_threshold"]
    smooth_l_threshold = secondary[2]["FRL-midas"]["smooth_l_threshold"]
    l_proxylabel = secondary[2]["FRL-midas"]["proxy_label"].to(torch.float32).tolist()

    logit_raw = secondary[2]["logit-confidence"]["logit"].to(torch.float32).tolist()
    logit_threshold = secondary[2]["logit-confidence"]["logit_threshold"].to(torch.float32).tolist()

    output_list = [",".join(map(str,item)) for item in zip(
      predicted_label,
      l2_dist,
      l2_hdthreshold,
      l2_proxylabels,
      cos_dist,
      cos_hdthreshold,
      cos_proxylabels,
      l_score,
      smooth_l_score,
      l_threshold,
      smooth_l_threshold,
      l_proxylabel,
      logit_raw,
      logit_threshold
      )]
    self.oobj.write("\n".join(output_list)+"\n")
  def end_of_deployment(self):
      self.oobj.close()

In [ ]:
ed = EdnaDeploy(config=["./GLAMOR/profiles/FNC/labeling/config_base.yml",  # logging, deployment, crawler
                          model_variant,
                        "./GLAMOR/profiles/FNC/experts/configs/plugin_base.yml"],
                config_inject = [
                    ("SAVE.MODEL_CORE_NAME", model_core_name),
                    ("SAVE.MODEL_BACKBONE", model_backbone),
                    ("SAVE.MODEL_QUALIFIER", model_qualifier),
                    ("SAVE.DRIVE_BACKUP", True),
                    ("SAVE.LOG_BACKUP", False),
                    ("MODEL.MODEL_BASE", model_backbone),
                    ("SAVE.SAVE_FREQUENCY", 5), # To ensure nothing actually gets saved, since we only run for 1 epoch
                    ("DEPLOYMENT.EPOCHS", 1)
                ])

ed.cfg.DEPLOYMENT.OUTPUT_ARGS["label_file_name"] = tweet_file + "-" + ed.saveMetadata.MODEL_SAVE_FOLDER+".csv"
ed.cfg.DEPLOYMENT.DATAREADER.CRAWLER_ARGS["azfile"] = "%s-extended.json.gz"%tweet_file
# No masking for everything, so we will not edit it here. 


ed.add("./GLAMOR/profiles/FNC/fnc-extended-labeling-crawler.py")
ed.addDeploymentClass(LabelingDeployment)

In [ ]:
ed.apply()

In [ ]:
ed.deploy()

In [ ]:
ed.model.plugins["FastKMP-l2"].proxies

In [ ]:
import pandas as pd
df=pd.read_csv("tweets-2020-01-22-fnc-expert-efnd-nomask-v1-albert-base-v2-kagglefn_short.csv")

In [ ]:
df

## Exploring

# ------------------------   Extras   -----------------------------------------

# Generating plugins

### CHECKLIST:

- is the `model_config` correct?
- is the `deploy_config` correct?
- is the `model_plugins` correct? This should not need to change for any of them.

- is the `model_functions` correct? Cross-reference to `model_config`!

- is the `dataloader_mode` correct?
- is the `batch_size` correct?

In [ ]:
GENERATING_PLUGINS = True
model_config = "./GLAMOR/profiles/MiDAS/expert-experiments/midas-cmumiscov-2.yml"
deploy_config = "./GLAMOR/profiles/NELA/nela-covid-deploy.yml"
model_plugins = "./GLAMOR/profiles/NELA/plugins.yml"
model_functions = ["./GLAMOR/profiles/MiDAS/midas-dataset.py",
                    "./GLAMOR/profiles/MiDAS/midas-expert.py"]
                    
dataloader_mode = "train" # CHANGE THIS IF DEBUGGING
batch_size = 128          # CHANGE THIS IF TOO SLOW OR CRASHING
# EXTRAS
deployment_functions = "./GLAMOR/profiles/NELA/deploy-plugin-usage.py"  # use generation.py is generating plugins
epochs = 1                # CHANGE THIS IF GENERATING PLUGINS ----> 6       ELSE, use 1 epoch
ignore_plugins = []       # CHANGE THIS IF PLUGINS NEED RESETTING ----> ["KMP-l2", "KMP-cos", "RL-midas"]
if GENERATING_PLUGINS:
  ignore_plugins = ["KMP-l2", "KMP-cos", "RL-midas"]
  epochs = 6
  deployment_functions =  "./GLAMOR/profiles/NELA/deploy-plugin-generation.py"

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import ednaml
from ednaml.core import EdnaDeploy
from ednaml.plugins.KMeansProxy import KMeansProxy
torch.__version__

In [ ]:
ed = EdnaDeploy(config=model_config, deploy=[deploy_config, model_plugins] , dataloader_mode = dataloader_mode)

We make sure we have the correct `DATAREADER`. Here we use the one from `EXECUTION`, because it comes from the model we are loading, ensuring we are working with the model's training/testing data. 

We can also manually set `EPOCHS` here, for debugging or fast deployment editing purposes. Since deployments are ephemeral and not logged (for the time being), this is sufficient.

In [ ]:
ed.cfg.DEPLOYMENT.DATAREADER = ed.cfg.EXECUTION.DATAREADER
#
ed.cfg.DEPLOYMENT.DATAREADER.DATAREADER = "AlbertReader"
ed.cfg.TEST_TRANSFORMATION.BATCH_SIZE = batch_size
ed.cfg.DEPLOYMENT.EPOCHS = epochs
ed.cfg.DEPLOYMENT.PLUGIN.HOOKS = 'warmup'

If needed, go into the files and make sure the correct functions are registered with EdnaML, specifically the correct deployment plugin.

There is one deployment plugin for writing to an output, and one for not writing to an output (for generating the plugins and doing nothing with the actual returns.

**`NOTE: potential bug. midas-dataset also contains MiDASGenerator now, but this part NEEDS to use AlBERTReader. Solution. Deregister MiDASGenerator from midas-datasert. Solution: integrated MiDASGenerator and AlbertReader with datalabels.`**

In [ ]:
ed.add(model_functions)
ed.add(deployment_functions)

In [ ]:
ed.apply()

In [ ]:
ed.deploy(ignore_plugins = ignore_plugins)

In [ ]:
ed.model.plugins['RL-midas'].save()

In [ ]:
ed.model.plugins['RL-midas'].save()

# Cross Validation Experiments

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

## Testing NELA/Fakeddit on MiDAS Datasets

In [ ]:
# NELA
cfg = "./GLAMOR/profiles/NELA/nela-covid-v1.yml"
model_file = "./GLAMOR/profiles/NELA/nela.py"
# Fakeddit
#cfg = "./GLAMOR/profiles/Fakeddit/fakeddit-v3.yml"
#model_file = "./GLAMOR/profiles/Fakeddit/fakeddit.py"

In [ ]:
data_folder = "Data"
datasets_folders = glob.glob(os.path.join(data_folder, "*"))
datasets_folders = [os.path.basename(item) for item in datasets_folders if os.path.basename(item) not in ["coaid_tweets", "recov", "nela-elections-2020", "all_train.tsv", "all_test_public.tsv", "nela-covid-2020", "nela-gt-2020", "all_validate.tsv"]]
datasets_folders

In [ ]:
# set up the model
eml = EdnaML(config=cfg, mode="test", test_only = True, logger_save_name="cd-eval")
eml2 = EdnaML(config="./GLAMOR/profiles/MiDAS/encoder-experiments/midas-tsne.yml", 
                mode="test", test_only = True, logger_save_name="cd-eval")  # weights path should be automatically inferred!!! :)

eml.cfg.EXECUTION.DATAREADER = eml2.cfg.EXECUTION.DATAREADER
eml.cfg.EXECUTION.DATAREADER.DATAREADER = "AlbertReader"
eml.cfg.EXECUTION.DATAREADER.CRAWLER_ARGS["include"] = datasets_folders
# So we have model loaded with correct parameters

eml.add(model_file) #add model
eml.add("./GLAMOR/profiles/MiDAS/midas-dataset.py") # replace crawler
eml.add("./GLAMOR/profiles/MiDAS/midas-trainer.py") # replace trainer

In [ ]:
eml.apply()

In [ ]:
preds, labels, classes, logits = eml.eval()

In [ ]:
from sklearn.metrics import f1_score
import numpy as np

print ("Domain accuracies for %s"%str(eml.cfg.SAVE.MODEL_QUALIFIER))
for idx, dlabel in enumerate(range(max(labels[1]).item()+1)):
  acc = torch.mean((preds[labels[1]==dlabel] == labels[0][labels[1]==dlabel]).float())
  micro_fscore = np.mean(f1_score(labels[0][labels[1]==dlabel], preds[labels[1]==dlabel], average="micro"))
  weighted_fscore = np.mean(f1_score(labels[0][labels[1]==dlabel], preds[labels[1]==dlabel], average="weighted"))
  print ("\t Domain {0}\t {1:0.3f} \t{2:0.3f} \t{3:0.3f}\t\t{4}".format(datasets_folders[idx], acc, micro_fscore, weighted_fscore, torch.sum(labels[1]==dlabel).item()))

## Testing MIDAS Models on NELA/Fakeddit

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
expert_config = "./GLAMOR/profiles/MiDAS/expert-experiments/midas-covrumor-1.yml"
"""
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-kagglefnlong-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-covrumor-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-covidfn-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-covid_cq-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-cov19fntitle-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-cov19fntext-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-coaid_news-1.yml"
"./GLAMOR/profiles/MiDAS/expert-experiments/midas-cmumiscov-2.yml"
"""

In [ ]:
# NELA
#cfg = "./GLAMOR/profiles/NELA/nela-covid-v1.yml"
#crawler_file = "./GLAMOR/profiles/NELA/nela.py"
# Fakeddit
cfg = "./GLAMOR/profiles/Fakeddit/fakeddit-v3.yml"
crawler_file = "./GLAMOR/profiles/Fakeddit/fakeddit.py"

In [ ]:
# set up the model
eml = EdnaML(config=expert_config, mode="test", test_only = True, logger_save_name="cd-eval")
eml2 = EdnaML(config=cfg, 
                mode="test", test_only = True, logger_save_name="cd-eval")  # weights path should be automatically inferred!!! :)

eml.cfg.EXECUTION.DATAREADER = eml2.cfg.EXECUTION.DATAREADER
# So we have model loaded with correct parameters

eml.add(crawler_file) # add crawler/trainer
eml.add("./GLAMOR/profiles/MiDAS/midas-expert.py") #replace model

In [ ]:
eml.apply()

In [ ]:
preds, labels, classes, logits = eml.eval()

In [ ]:
labels[labels!=0] = 1

In [ ]:
accuracy = (preds != labels).sum().float() / float(labels.size(0))
print("\tAccuracy: {:.3%}".format(accuracy))

## NELA <-> Fakeddit cross validation

In [ ]:
import torch
import ednaml
import glob, os
from ednaml.core import EdnaDeploy, EdnaML
torch.__version__

In [ ]:
# NELA
b_cfg = "./GLAMOR/profiles/NELA/nela-covid-v1.yml"
b_crawlerfile = "./GLAMOR/profiles/NELA/nela.py"
# Fakeddit
cfg = "./GLAMOR/profiles/Fakeddit/fakeddit-v3.yml"
model_file = "./GLAMOR/profiles/Fakeddit/fakeddit.py"

In [ ]:
# set up the model
eml = EdnaML(config=cfg, mode="test", test_only = True, logger_save_name="cd-eval")
eml2 = EdnaML(config=b_cfg, 
                mode="test", test_only = True, logger_save_name="cd-eval")  # weights path should be automatically inferred!!! :)

eml.cfg.EXECUTION.DATAREADER = eml2.cfg.EXECUTION.DATAREADER
# So we have model loaded with correct parameters

eml.add(b_crawlerfile) #add crawler/trainer
eml.add(model_file) # replace model

In [ ]:
eml.apply()

In [ ]:
preds, labels, classes, logits = eml.eval()

#### Converting the fakeddit true labels to other 1/0 versions

In [ ]:
labels[labels!=0] = 1

In [ ]:
accuracy = (preds == labels).sum().float() / float(labels.size(0))
print("\tAccuracy: {:.3%}".format(accuracy))

In [ ]:
!rm -rf -- test-datashard-artifacts/

#### Converting the Fakeddit predictions to other versions

In [ ]:
preds[preds!=0] = 1

In [ ]:
accuracy = (preds == labels).sum().float() / float(labels.size(0))
print("\tAccuracy: {:.3%}".format(accuracy))